### Build a SuperDove Constellation in Sedaro

This notebook exercises functionality of deleting any agents with outdated TLEs in the [DEMO] Planet SuperDove Scenario and repopulating the scenario with new agents using current TLEs through the Sedaro API.


In [ ]:
import json
import requests

from sedaro import SedaroApiClient

#### Important: Read Before Running

This notebook makes changes to a scenario branche indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [ ]:
# Settings
with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = config['SUPERDOVE']['SCENARIO_BRANCH_ID']      # ID of the scenario branch
HOST = config['HOST']                                               # Sedaro instance URL
BASE_AGENT_ID = 'NT-KoZFSELKK8eomP3lkV'

#### Instantiate Client

Instantiate the `SedaroApiClient` with our `API_KEY` and `HOST`, and fetch the desired branch.


In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)

scenario_branch = sedaro.scenario(SCENARIO_BRANCH_ID)

### Delete satelllites with old TLEs

This is a cleanup step to remove any satellites that were created in a previous run of this notebook. This is not necessary if you are running this notebook for the first time.


In [ ]:
delete_ids = [agent_id for agent_id in scenario_branch.TemplatedAgent.get_all_ids() if agent_id != BASE_AGENT_ID]

if not delete_ids:
    print('No blocks were deleted')
else:
    scenario_branch.crud(delete=delete_ids)
    print(f'Deleted {len(delete_ids)} spacecraft digital twin agents.')

### Fetch Constellation TLEs


In [ ]:
# Load TLE file
url = 'https://ephemerides.planet-labs.com/planet_mc.tle'
tle_file = requests.get(url).text.splitlines()
tles = [[tle_file[x].strip()] + tle_file[x+1:x+3] for x in range(0, len(tle_file), 3)]  # split into 3 line TLEs
print(f"Fetched {len(tles)} TLEs.")

### Create new agents with new TLEs


In [ ]:
base_agent = scenario_branch.Agent.get(BASE_AGENT_ID)

orbits = [
    {
        **base_agent.kinematics.data,
        'initialStateDefParams': {'tle': '\n'.join(tle[1:3])},
        'id': f'$-{tle[0]}-orbit',
    } for tle in tles
]

agents = [
    {
        **base_agent.data,
        'id': f'$-{tle[0]}-agent',
        'name': tle[0][2:],
        'kinematics': f'$-{tle[0]}-orbit',
    } for tle in tles
]

scenario_branch.crud(blocks=orbits + agents)
print(f'Created {len(tles)} orbits from TLEs and {len(tles)} spacecraft digital twin agents.')

### Update Simulation Clock


In [ ]:
from datetime import datetime, UTC

mjd_days = (datetime.now(UTC) - datetime(year=1858, month=11, day=17, tzinfo=UTC)).days
scenario_branch.ClockConfig.get_first().update(
    startTime=mjd_days,
    stopTime=mjd_days + 1
)